In [1]:
from PIL import Image
import numpy as np
import os
import cv2
import pickle

In [2]:
def _plot_point_(image, points):
    color_list = [
        (255, 0, 0),  # 红色
        (0, 255, 0),  # 绿色
        (0, 0, 255),  # 蓝色
        (255, 255, 0),  # 黄色
        (255, 0, 255),  # 粉红色
        (0, 255, 255)  # 青色
    ]
    for i,point in enumerate(points):
        cv2.circle(image, point.astype(np.int32), 7, color_list[i], -1)  # 绘制色点，半径为5
    return image


def load_result(pred_dir):
    with open(pred_dir,'rb') as f:
        result = pickle.load(f)
    pred = []
    label = []
    image = []
    for res in result:
        pred.append(res['pred_instances']['keypoints'])
        label.append(res['gt_instances']['keypoints'])
        image.append(res['img_path'])
    return pred,label,image

#### pred point

In [30]:
model_list = ['hrformer','hrnet_w48','pvt','pvtv2','SCNet','simple','swin']
for model in model_list:
    pred_dir = f'../work_dirs/{model}/pred.pkl'
    save_root = f'../work_dirs/{model}/pred_point/'
    img_root =  '/media/HDD/hln/dataset/HipJoint_mmseg/images/'
    try:
        os.makedirs(save_root)
    except:
        pass
    pred_list, label_list, image_list = load_result(pred_dir)
    for id in range(len(pred_list)):
        pred_point = pred_list[id].squeeze(0)
        label_point = label_list[id].squeeze(0)
        img_dir = image_list[id]
        img_name = os.path.basename(img_dir)
        img_dir = img_root+img_name
        # 读取原图
        img = Image.open(img_dir).convert('RGB')
        img = np.array(img)
        # 画图
        img_point = _plot_point_(image=img, points=pred_point)
        img_point = Image.fromarray(img_point)
        # 保存
        save_name = os.path.basename(img_dir)[:-3]+'png'
        save_dir = save_root+save_name
        img_point.save(save_dir)

#### pred heat

In [18]:
model_list = ['hrformer','hrnet_w48','pvt','pvtv2','SCNet','simple','swin','darkpose','hourglass']
for model in model_list:
    pred_root = f'../work_dirs/{model}/work_dirs/{model}/'
    save_root = f'../work_dirs/{model}/heat_point/'
    img_root =  '/media/HDD/hln/dataset/HipJoint_mmseg/images/'
    try:
        os.makedirs(save_root)
    except:
        pass
    for filename in os.listdir(pred_root):
        pred_dir = pred_root+filename
        img_dir = img_root+filename[:-6]+'.jpg'
        save_dir = save_root+filename[:-6]+'.png'
        pred_img = Image.open(pred_dir).convert('RGB')
        img = Image.open(img_dir).convert('RGB')
        pred_img = np.array(pred_img)
        H2,W,C = pred_img.shape
        H = H2//2
        pred_heat = pred_img[H:]
        pred_heat = Image.fromarray(pred_heat)
        pred_heat.save(save_dir)



In [6]:
model_list = ['hrformer','hrnet_w48','pvt','pvtv2','SCNet','simple','swin','darkpose','hourglass']
model_list = ['hrformer']
for model in model_list:
    pred_dir = f'../work_dirs/{model}/pred.pkl'
    save_root = f'../work_dirs/{model}/heat_point/'
    img_root =  '/media/HDD/hln/dataset/HipJoint_mmseg/images/'
    try:
        os.makedirs(save_root)
    except:
        pass
    pred_list, label_list, image_list = load_result(pred_dir)
    for id in range(len(pred_list)):
        pred_point = pred_list[id].squeeze(0)
        label_point = label_list[id].squeeze(0)
        img_dir = image_list[id]
        img_name = os.path.basename(img_dir)
        img_dir = img_root+img_name
        # 处理关键点
        pred_point = np.round(pred_point).astype(np.int32)
        # 读取原图
        img = Image.open(img_dir).convert('RGB')
        img = np.array(img)
        # 创建热力图
        def generate_heatmap(heatmap, point, sigma):
            x,y = point
            heatmap[y][x] = 1
            heatmap = cv2.GaussianBlur(heatmap, sigma, 5)
            am = np.amax(heatmap)
            heatmap /= am / 255
            heatmap[y][x]+=255
            heatmap/=2
            return heatmap
        # 画图
        heatmap = np.zeros((pred_point.shape[0],img.shape[0],img.shape[1]))
        for i in range(heatmap.shape[0]):
            heatmap[i] = generate_heatmap(heatmap[i], pred_point[i], sigma=(15,15))
        heatmap = heatmap.sum(0)
        heatmap[heatmap>=255] = 255
        heatmap[heatmap<0]=0
        heatmap = cv2.applyColorMap(heatmap.astype(np.uint8), cv2.COLORMAP_JET)
        result = heatmap * 0.6 + img * 0.4
        # 保存
        save_name = os.path.basename(img_dir)[:-3]+'png'
        save_dir = save_root+save_name
        cv2.imwrite(save_dir, result)


#### label point

In [3]:
pred_dir = f'../work_dirs/hrformer/pred.pkl'
save_root = f'../work_dirs/label/pred_point/'
img_root =  '/media/HDD/hln/dataset/HipJoint_mmseg/images/'
try:
    os.makedirs(save_root)
except:
    pass
pred_list, label_list, image_list = load_result(pred_dir)
for id in range(len(pred_list)):
    pred_point = pred_list[id].squeeze(0)
    label_point = label_list[id].squeeze(0)
    img_dir = image_list[id]
    img_name = os.path.basename(img_dir)
    img_dir = img_root+img_name
    # 读取原图
    img = Image.open(img_dir).convert('RGB')
    img = np.array(img)
    # 画图
    img_point = _plot_point_(image=img, points=label_point)
    img_point = Image.fromarray(img_point)
    # 保存
    save_name = os.path.basename(img_dir)[:-3]+'png'
    save_dir = save_root+save_name
    img_point.save(save_dir)

#### label heatmap

In [10]:
pred_dir = f'../work_dirs/hrformer/pred.pkl'
save_root = f'../work_dirs/label/heat_point/'
img_root =  '/media/HDD/hln/dataset/HipJoint_mmseg/images/'
try:
    os.makedirs(save_root)
except:
    pass
pred_list, label_list, image_list = load_result(pred_dir)
for id in range(len(pred_list)):
    pred_point = pred_list[id].squeeze(0)
    label_point = label_list[id].squeeze(0)
    img_dir = image_list[id]
    img_name = os.path.basename(img_dir)
    img_dir = img_root+img_name
    # 读取原图
    img = Image.open(img_dir).convert('RGB')
    img = np.array(img)
    # 画图
    def generate_heatmap(heatmap, point, sigma):
        x,y = point
        heatmap[y][x] = 1
        heatmap = cv2.GaussianBlur(heatmap, sigma, 5)
        am = np.amax(heatmap)
        heatmap /= am / 255
        heatmap[y][x]+=255
        heatmap/=2
        return heatmap
    # 画图
    label_point = label_point.astype(np.int32)
    heatmap = np.zeros((label_point.shape[0],img.shape[0],img.shape[1]))
    for i in range(heatmap.shape[0]):
        heatmap[i] = generate_heatmap(heatmap[i], label_point[i], sigma=(19,19))
    heatmap = heatmap.sum(0)
    heatmap[heatmap>=255] = 255
    heatmap[heatmap<0]=0
    heatmap = cv2.applyColorMap(heatmap.astype(np.uint8), cv2.COLORMAP_JET)
    result = heatmap * 0.5 + img * 0.5
    # 保存
    save_name = os.path.basename(img_dir)[:-3]+'png'
    save_dir = save_root+save_name
    cv2.imwrite(save_dir, result)

#### 迁移文件夹

In [19]:
import os
import shutil

# 定义目录A和目录B的路径
dir_source = '/home/huliangni/Segmentation/mmpose/HipJoint/work_dirs/'
dir_des = '/home/huliangni/TMI/landmark_visual/'

model_list =  ['hrformer','hrnet_w48','pvt','pvtv2','SCNet','simple','swin','darkpose','hourglass','label']

# 定义要复制的子文件夹列表
subfolders = ['heat_point', 'pred_point']

for model in model_list:

    # 在目录B中创建hrnet文件夹的路径
    dir_des_model = os.path.join(dir_des, model)

    # 如果hrnet文件夹不存在，则创建它
    if not os.path.exists(dir_des_model):
        os.makedirs(dir_des_model)

    # 遍历子文件夹列表并复制每个文件夹
    for subfolder in subfolders:
        # 源文件夹路径
        source = os.path.join(dir_source, model, subfolder)
        # 目标文件夹路径
        destination = os.path.join(dir_des_model, subfolder)
        
        # 如果源文件夹存在，则复制到目标路径
        if os.path.exists(source):
            shutil.copytree(source, destination)
        else:
            print(f"Source folder {source} does not exist, skipping.")

    print(f"Copy {model} complete.")

Copy hrformer complete.
Copy hrnet_w48 complete.
Copy pvt complete.
Copy pvtv2 complete.
Copy SCNet complete.
Copy simple complete.
Copy swin complete.
Copy darkpose complete.
Copy hourglass complete.
Copy label complete.
